In [1]:
# Download a static FFmpeg build and add it to PATH.
%run 'util/load-ffmpeg.ipynb'

input_file = 'input.mp4'

print('Done!')

/usr/bin/ffmpeg
Done!


In [2]:
from IPython.display import HTML
HTML(
"""
<h3>Input video</h3>
<video muted loop controls>
  <source src="%s" type="video/mp4">
</video>""" % input_file)

In [3]:
# Plot an interactive visualization of the frame size of the input video.

output=!ffprobe -hide_banner -v error -select_streams v:0 -show_frames {input_file}
    
import pandas as pd
import numpy as np

def parseOutput(output_lines):
    frames = []
    current_frame = {}
    for line in output_lines:
        if line == '[/FRAME]':
            frames.append(current_frame)
            current_frame = {}
        elif line != '[FRAME]':
            pair = line.split('=')
            current_frame[pair[0]] = pair[1]
    return frames

def countByFrameType(frames):
    count = {}
    for frame in frames:
        t = frame['pict_type']
        count[t] = (count[t] + 1) if t in count else 1
    return count

def toPandasDf(frames): 
    dict_obj = {
        'frame index': range(len(frames)),
        'size (bytes)': [int(f['pkt_size']) for f in frames],
        'type': [f['pict_type'] for f in frames]
    }

    return pd.DataFrame.from_dict(dict_obj)

frames = parseOutput(output)

import plotly.express as px
framesize_df = toPandasDf(frames)

fig = px.bar(framesize_df, x="frame index", y="size (bytes)", color="type")
fig.show()


ModuleNotFoundError: No module named 'plotly'

In [4]:
# Create a compressed output video.
output_file = 'output.mp4'
!ffmpeg -v error -i {input_file} -crf 26 -y {output_file}

# Calculate per frame PSNR, SSIM, VMAF. Caution: this may take a while.
metric_log = 'metric.log'
!rm {metric_log}
!ffmpeg -hide_banner -i output.mp4 -i {input_file} -filter_complex \
    libvmaf=psnr=1:ssim=1:log_path={metric_log}:log_fmt=json:model_path=model/vmaf_v0.6.1.pkl -f null -


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'output.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.03, start: 0.000000, bitrate: 687 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 551 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #1:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 t

In [4]:
# Plot the per frame PSNR/SSIM/VMAF.

import json
with open(metric_log) as f:
  data = json.load(f)

metric_dict = {
    'index': list(range(len(data['frames']))),
    'psnr': [f['metrics']['psnr'] for f in data['frames']],
    'ssim': [f['metrics']['ssim'] for f in data['frames']],
    'vmaf': [f['metrics']['vmaf'] for f in data['frames']]
}

from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=3, subplot_titles=['psnr', 'ssim', 'vmaf'])

fig.add_trace(
    go.Scatter(x=metric_dict['index'], y=metric_dict['psnr']),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=metric_dict['index'], y=metric_dict['ssim']),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=metric_dict['index'], y=metric_dict['vmaf']),
    row=1, col=3
)
fig.update_layout(showlegend=False)
fig.show()


NameError: name 'metric_log' is not defined

In [6]:
# Specify a frame index and show the source and compressed frames side by side.
# Frame number range for the sample video: [0, 239].
# To see a different frame, update frame_index and re-run the cell.

frame_index = 78 # Starting from 0

!ffmpeg -v error -i {input_file} -vf select="eq(n\,{frame_index+1})" -frames:v 1 -y source.png
!ffmpeg -v error -i {output_file} -vf select="eq(n\,{frame_index+1})" -frames:v 1 -y compressed.png

from IPython.display import HTML
import random
refresh = random.random()
HTML("""
<h3>Visual comparison at frame %d</h3>
<img src='source.png?foo=%f'></img>
<img src='compressed.png?foo=%f'></img>""" % (frame_index, refresh, refresh))

In [7]:
# Use the 'codecview' filter to visualize motion vectors. 
# More details: https://ffmpeg.org/ffmpeg-filters.html#codecview
#
!ffmpeg -v error -flags2 +export_mvs -i {output_file} -vf codecview=mv=pf+bf+bb -y mv.mp4

HTML('''
<h3>Visualize motion vectors</h3>
<video src='mv.mp4' loop controls autoplay/>
''')